In [9]:
# import libraries
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, ToPILImage, CenterCrop, RandomResizedCrop
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, resnet18, inception_v3
from torchvision.models.alexnet import model_urls
from torch.hub import load_state_dict_from_url


# misc
import os
import sys
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

In [2]:
# logging directory
LOG_DIR = Path('../../logs')
if not LOG_DIR.is_dir():
    LOG_DIR.mkdir()

# logging file
LOG_FILE_PATH = os.path.join(LOG_DIR, f"04_alexnet_transfer_learning_v2.log")

In [3]:
# logging configuration
logging.basicConfig(
    level=logging.INFO,
    datefmt="%a, %d %b %Y-%m-%d %H:%M:%S",
    format="[%(asctime)s.%(msecs)03d] %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(filename=LOG_FILE_PATH, mode="w"),
        logging.StreamHandler()
    ]
)

In [4]:
# check torch version
logging.info(f"Torch version: {torch.__version__}")

# check torchvision version
logging.info(f"Torchvision version: {torchvision.__version__}")

# created date
logging.info(f"Created date: 2023-07-07 00:14:35")

# modified date
logging.info(f"Modified date: {datetime.now().strftime('%Y-%m-%m %H:%M:%S')}")

# num of gpus
gpus = torch.cuda.device_count()
logging.info(f"Number of gpus: {gpus}")

# device
device = "cuda" if torch.cuda.is_available() else "cpu"
logging.info(f"Device: {device}")

[Sun, 30 Jul 2023-07-30 00:37:37.459] INFO - Torch version: 1.12.1
[Sun, 30 Jul 2023-07-30 00:37:37.460] INFO - Torchvision version: 0.13.1
[Sun, 30 Jul 2023-07-30 00:37:37.460] INFO - Created date: 2023-07-07 00:14:35
[Sun, 30 Jul 2023-07-30 00:37:37.460] INFO - Modified date: 2023-07-07 00:37:37
[Sun, 30 Jul 2023-07-30 00:37:37.495] INFO - Number of gpus: 1
[Sun, 30 Jul 2023-07-30 00:37:37.496] INFO - Device: cuda


In [6]:
# loading `alexnet` architecture
alex = alexnet(pretrained=False)
logging.info(alex)

[Sun, 30 Jul 2023-07-30 00:38:36.243] INFO - AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_f

#### Loading Weights

In [11]:
# https://pytorch.org/vision/stable/_modules/torchvision/models/alexnet.html
url = model_urls['alexnet']
url

'https://download.pytorch.org/models/alexnet-owt-7be5be79.pth'

In [12]:
# loading pre-trained weights
state_dict = load_state_dict_from_url(url, model_dir="pretrained", progress=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to pretrained/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [01:24<00:00, 2.89MB/s] 


In [17]:
# loading model
alex.load_state_dict(state_dict)

<All keys matched successfully>